# Phase 1 Notebook: `memo_store.py`

Target file: `execution/langgraph/memo_store.py`

Purpose: persist memo entries in a schema-first SQLite backend with run-scoped lookups.

## 3-Layer Context

- Layer 1 (Directive): memory should be deterministic and durable.
- Layer 2 (Orchestration): `graph.py` and `tools_registry.py` use this store for memo put/get.
- Layer 3 (Execution): SQLite operations are deterministic and schema-constrained.

In [ ]:
from pathlib import Path
import sys
import os

def bootstrap_repo_root() -> Path:
    cwd = Path.cwd().resolve()
    candidates = [cwd, *cwd.parents, Path('/home/nir/dev/agent_phase0')]
    for candidate in candidates:
        if (candidate / 'execution' / 'langgraph' / 'memo_store.py').exists():
            if str(candidate) not in sys.path:
                sys.path.insert(0, str(candidate))
            return candidate
    raise RuntimeError('Could not locate repo root for memo store notebook.')

repo_root = bootstrap_repo_root()
from dotenv import load_dotenv
load_dotenv(repo_root / ".env")
print('P1_PROVIDER =', os.getenv('P1_PROVIDER', 'ollama'))
print('repo_root =', repo_root)
print('kernel_python =', sys.executable)
if '/.venv/' not in sys.executable.replace('\\', '/'):
    print('WARNING: kernel is not the project .venv interpreter.')

## P0 vs P1 (Memo Persistence)

| Concern | Phase 0 (`tools/memoize.py`) | Phase 1 (`memo_store.py`) | Why this matters |
|---|---|---|---|
| Persistence model | direct file overwrite by key/path | structured SQLite table + index | queryable, auditable, and safer updates |
| Scope | global file path side-effects | run-scoped (`run_id`, `namespace`) | avoids cross-run contamination |
| Identity | raw value only | value hash + metadata | traceability and deterministic comparisons |

## Why this block exists: inspect schema + API

We inspect table setup and `put/get` signatures to verify the store is ready for future backend replacement (for example Postgres).

In [ ]:
import inspect
from execution.langgraph.memo_store import SQLiteMemoStore

print(inspect.getsource(SQLiteMemoStore._initialize_schema))
print(inspect.getsource(SQLiteMemoStore.put))
print(inspect.getsource(SQLiteMemoStore.get))

## Why this block exists: run-scoped round-trip demo

This validates that memo values are isolated per run and that hashes are stable for identical values.

In [ ]:
from execution.langgraph.memo_store import SQLiteMemoStore

store = SQLiteMemoStore('.tmp/notebook_demo_memo.db')
put_result = store.put(run_id='run-A', key='write_file:fib.txt', value={'n': 100, 'ok': True}, source_tool='write_file', step=4)
hit = store.get(run_id='run-A', key='write_file:fib.txt')
miss = store.get(run_id='run-B', key='write_file:fib.txt')
put_result, hit, miss

In [ ]:
assert hit.found is True
assert miss.found is False
assert hit.value_hash == put_result.value_hash
print('memo store assertions passed')

## Takeaways

- P1 memoization is now schema-backed and run-scoped.
- This store is backend-replaceable while preserving interface shape.
- P0 behavior is preserved conceptually but hardened operationally.
- Next notebook: `execution/notebooks/p1_checkpoint_store.ipynb`.